# Latauslaskuri

Tällä laskurilla voit laskea, kuinka paljon sähköauton (tai lataushybridin)
lataus maksaa pörssisähköä käyttäen.
Kulutusdata annetaan minuuttikohtaisesti CSV-muodossa, ja pörssisähkön
hintatiedot ladataan automaattisesti latausjaksolle.
Laskussa huomioidaan pörssisähkön hinnan lisäksi kiinteät kustannukset eli
pörssisähkösopimuksen mukainen marginaali, sähkönsiirto sekä sähkövero.

In [1]:
from datetime import datetime
import os
import time

import numpy as np
import pandas as pd
import requests

Määritetään sähkön kiinteät kustannukset senteissä per kilowattitunti
(snt kWh⁻¹).

In [2]:
MARGINAALI = 0.49
SIIRTO = 3.0
SAHKOVERO = 2.82752

kiinteat = MARGINAALI + SIIRTO + SAHKOVERO

### Kulutuksen lukeminen CSV-tiedostoista

Alla olevassa solussa kulutustiedot luetaan datakansiossa olevista
CSV-tiedostoista pandas DataFrameen.
Tämän jälkeen minuuttikohtainen kulutusdata muutetaan varttikohtaiseksi
summaamalla, ja datan yksiköt muunnetaan wattitunneista (Wh) kilowattitunteihin
(kWh) jakamalla varttikohtainen kulutuslukeman tuhannella.
Datasta poistetaan latausaseman valmiustilan kulutus asettamalla kaikki alle
1 Wh:n kulutuslukeman nollaan.

Shellyltä saatava CSV-tiedosto sisältää kulutusdatan lisäksi myös tiedot
sähköntuotannosta.
Nämä jätetään lukematta asettamalla `max_rows`-parametrille arvoksi 60.

In [3]:
minuuttikulutus = pd.DataFrame()

data_dir = './data'
directory = os.fsencode(data_dir)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.csv'):
        ajat, kulutukset = np.loadtxt(
            os.path.join(data_dir, filename),
            dtype='str',
            delimiter=',',
            skiprows=2,
            unpack=True,
            max_rows=60,
        )
        
        ajat = np.array(
            [datetime.strptime(i.strip(), '%d/%m/%Y %H:%M') for i in ajat],
        )
        kulutukset = np.array(
            [float(i) for i in kulutukset],
        )
        minuuttikulutus = pd.concat(
            [minuuttikulutus, pd.DataFrame(kulutukset, index=ajat)],
        )

minuuttikulutus.columns = ['kulutus (kWh)']

minuuttikulutus.loc[minuuttikulutus.loc[:, 'kulutus (kWh)'] < 1] = 0

varttikulutus = minuuttikulutus.resample(
    pd.Timedelta(minutes=15),
).sum().div(1e3)

Shellyn kulutusdatassa on usein minuutin tai kahden aukkoja.
Vaikka aukot ovat lyhyitä, ne kuitenkin vaikuttavat lopulliseen käytetyn
sähkön määrään ja näin ollen latauksen kokonaishintaan.

Tässä lyhyiden aukkojen yli interpoloidaan, jolloin saadaan laskettua
todellinen toteutunut latauksen hinta.

In [4]:
minuuttikulutus = minuuttikulutus.sort_index().replace(0, np.nan)

lukemat = minuuttikulutus.loc[:, 'kulutus (kWh)'].notnull()
aukkojen_reunat = lukemat.ne(lukemat.shift()).cumsum()
aukon_koko = minuuttikulutus.groupby(
    [aukkojen_reunat, minuuttikulutus.loc[:, 'kulutus (kWh)'].notnull()],
)['kulutus (kWh)'].transform('size').where(
    minuuttikulutus.loc[:, 'kulutus (kWh)'].isnull(),
)
    
minuuttikulutus_interp = minuuttikulutus.interpolate(
    limit_area='inside',
    method='linear',
).mask(aukon_koko > 3)
varttikulutus_interp = minuuttikulutus_interp.resample(
    pd.Timedelta(minutes=15),
).sum().div(1e3)

### Latauksen hinnan laskeminen

Seuraavaksi luodaan uusi pandas DataFrame, jossa indeksinä on varttikohtainen
aika, ja sarakkeina pörssisähkön hinta (yksiköissä snt kWh⁻¹), sähkönkulutus
(yksiköissä kWh), ja kulutuskohtainen hinta (yksiköissä snt).

Jotta puuttuvien sähkönkulutuslukemien vaikutus nähdään suoraan, luodaan
erilliset DataFramet alkuperäiselle ja interpoloidulle datalle.

In [5]:
latauksen_hinta = pd.DataFrame(
    index=varttikulutus.index,
    columns=['pörssi (snt/kWh)', 'kulutus (kWh)', 'hinta (snt)'],
)
latauksen_hinta_interp = pd.DataFrame(
    index=varttikulutus_interp.index,
    columns=['pörssi (snt/kWh)', 'kulutus (kWh)', 'hinta (snt)'],
)

Tämän jälkeen ladataan pörssisähkön varttikohtaiset hinnat tarkastellulle
aikavälille.
Tässä tulee huomioida, että kulutusdatan aikavyöhyke on Suomen aikavyöhyke eli
UTC+2 talvi- ja UTC+3 kesäajalle, kun taas pörssisähkön hintatiedot annetaan
UTC-ajassa.
Tämän takia `latauksen_hinta` DataFramen indeksistä otettavat ajat on hintoja
ladattaessa muutettava UTC-aikaan, jotta oikea pörssisähkön hinta kohdistuu
oikealle kulutuslukemalle.

In [6]:
aikavyohyke_sek = time.localtime().tm_gmtoff

for aika in latauksen_hinta.index:
    aika_utc = aika - pd.Timedelta(seconds=aikavyohyke_sek)
    aika_str = aika_utc.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    hinta = requests.get(
        f'https://api.porssisahko.net/v2/price.json?date={aika_str}',
        ).json()
    latauksen_hinta.loc[aika, 'pörssi (snt/kWh)'] = hinta['price']
    latauksen_hinta_interp.loc[aika, 'pörssi (snt/kWh)'] = hinta['price']

Lopuksi lasketaan auton latauksen varttikohtainen hinta lisäämällä pörssisähkön
hintoihin kiinteät kulut ja kertomalla kulutuslukemat näillä summilla.
Latauksen kokonaishinta saadaan sitten summaamalla varttikohtaiset hinnat
yhteen.
Varttikohtaiset hinnat ovat senteissä, joten lopputulos muutetaan vielä euroiksi
jakamalla lopullinen summa sadalla.

Ensin lasketaan latauksen hinta Shellyltä saadulle datalle ilman mahdollisten
aukkojen interpolointia, sen jälkeen sama interpoloidulle datalle.
Jos datassa ei ole aukkoja, näiden tulisi tietysti olla yhtä suuret.

In [7]:
latauksen_hinta.loc[varttikulutus.index, 'kulutus (kWh)'] = (
    varttikulutus.loc[:, 'kulutus (kWh)']
)
latauksen_hinta.loc[:, 'hinta (snt)'] = (
    (latauksen_hinta.loc[:, 'pörssi (snt/kWh)'] + kiinteat)
    *latauksen_hinta.loc[:, 'kulutus (kWh)']
)

latauksen_hinta_interp.loc[varttikulutus_interp.index, 'kulutus (kWh)'] = (
    varttikulutus_interp.loc[:, 'kulutus (kWh)']
)
latauksen_hinta_interp.loc[:, 'hinta (snt)'] = (
    (latauksen_hinta_interp.loc[:, 'pörssi (snt/kWh)'] + kiinteat)
    *latauksen_hinta_interp.loc[:, 'kulutus (kWh)']
)

Lopuksi tulostetaan latauksen aloitus- ja lopetusajankohdat, ladatun sähkön
kokonaismäärä sekä latauksen kokonaishinta.

Huom! Tässä tulostetaan interpoloidut lopputulokset, alkuperäisiä ei erikseen
tulosteta, mutta ne löytyvät myös muistista jos alkuperäisiä ja interpoloituja
tuloksia halutaan verrata.

In [8]:
latauksen_aloitus = minuuttikulutus_interp.first_valid_index()
latauksen_lopetus = minuuttikulutus_interp.last_valid_index()
ladattu_sahko = latauksen_hinta_interp.loc[:, 'kulutus (kWh)'].sum()
kokonaishinta = latauksen_hinta_interp.loc[:, 'hinta (snt)'].div(1e2).sum()

print(f'Lataus alkoi {latauksen_aloitus}.')
print(f'Lataus päättyi {latauksen_lopetus}.')
print(f'Autoon ladattiin {ladattu_sahko:.2f} kWh sähköä.')
print(f'Lataus maksoi kokonaisuudessaan {kokonaishinta:.2f}€.')

Lataus alkoi 2026-02-19 00:01:00.
Lataus päättyi 2026-02-19 02:44:00.
Autoon ladattiin 6.76 kWh sähköä.
Lataus maksoi kokonaisuudessaan 1.29€.
